In [126]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

In [3]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df1=pd.DataFrame(df[0])
df1.head(10)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [4]:
oba= df1[df1['Borough']=='Not assigned'].index
df1.drop(oba, inplace=True)
df1['Neighborhood']= df1[['Postal code','Borough','Neighborhood']].groupby(['Postal code','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
df1.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Contrarly to the assigment description, I noticed that all the Borough has an assigned Neighborhood

In [5]:
url="http://cocl.us/Geospatial_data"
df3 = pd.read_csv(url)

df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df4=df1.set_index('Postal code').join(df3.set_index('Postal Code'))
df5=df4.reset_index()
df5.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [33]:
dfTor=df5[df5.Borough.str.contains('Toronto')]
dfTor.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [31]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [42]:
map_Toronto = folium.Map(location=[43.52, -79.35], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfTor['Latitude'], dfTor['Longitude'], dfTor['Borough'], dfTor['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [43]:
CLIENT_ID = '
CLIENT_SECRET = ' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UHWPBE2R0GG33ATQWYX4WCTTNKF5GA535WA0LTMIG5RXBDJO
CLIENT_SECRET:KOMD1Z0QABT2H0OJXNJRXAYHI341ZPWB2TF1C5FWW1DEUZT1


In [50]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
dfToronto_venues = getNearbyVenues(names=dfTor['Neighborhood'],
                                   latitudes=dfTor['Latitude'],
                                   longitudes=dfTor['Longitude']
                                  )


Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst  Quay / South Niagara / Island airport
Rosed

In [54]:
dfToronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park / Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park / Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park / Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [56]:
dfToronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
Brockton / Parkdale Village / Exhibition Place,23,23,23,23,23,23
Business reply mail Processing CentrE,15,15,15,15,15,15
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,16,16,16,16,16,16
Central Bay Street,77,77,77,77,77,77
Christie,18,18,18,18,18,18
Church and Wellesley,81,81,81,81,81,81
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [67]:
Toronto_onehot = pd.get_dummies(dfToronto_venues[['Venue Category']], prefix="", prefix_sep="")


In [68]:
Toronto_onehot['Neighborhood'] = dfToronto_venues['Neighborhood'] 

In [134]:
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(2)

,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Business reply mail Processing CentrE,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.066667
3,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012987,0.000000,...,0.000000,0.00,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.00,0.012987
5,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.012346,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012346,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012346,0.012346,0.00,0.024691
7,Commerce Court / Victoria Hotel,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.000000,...,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
8,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [76]:
from sklearn.cluster import KMeans
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_[0:1000] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 4, 0,
       0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [91]:
Toronto_grouped.insert(0, 'Neighborhood Cluster', kmeans.labels_)


In [92]:
Toronto_merged = dfTor

Toronto_merged = Toronto_merged.join(Toronto_grouped.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head(20000) # check the last columns

,Postal code,Borough,Neighborhood,Latitude,Longitude,Neighborhood Cluster,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.022222
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.00,0.030303
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.000000,0.00,0.000000
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0.000000,0.0000,0.0000,0.0000,...,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.012987,0.000000,0.000000,0.012987,0.000000,0.000000,0.00,0.012987
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
30,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000
31,M6H,West Toronto,Dufferin / Dovercourt Village,43.669005,-79.442259,0,0.000000,0.0000,0.0000,0.0000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [100]:
map_Torontocl = folium.Map(location=[43.52, -79.35], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Neighborhood Cluster']):
    label = folium.Popup(str(poi) + ' Neighborhood Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_Torontocl)
       
map_Torontocl

In [257]:
Toronto_merged1=Toronto_merged.drop(columns=['Postal code','Borough','Neighborhood','Latitude','Longitude','Neighborhood Cluster'])

Cluster0=Toronto_merged1[Toronto_merged['Neighborhood Cluster']==0]
Cluster1=Toronto_merged1[Toronto_merged['Neighborhood Cluster']==1]
Cluster2=Toronto_merged1[Toronto_merged['Neighborhood Cluster']==2]
Cluster3=Toronto_merged1[Toronto_merged['Neighborhood Cluster']==3]
Cluster4=Toronto_merged1[Toronto_merged['Neighborhood Cluster']==4]

In [283]:
df0=pd.DataFrame(Cluster0.sum(axis=0)).sort_values(by = 0 , ascending=False)
df1=pd.DataFrame(Cluster1.sum(axis=0)).sort_values(by = 0 , ascending=False)
df2=pd.DataFrame(Cluster2.sum(axis=0)).sort_values(by = 0 , ascending=False)
df3=pd.DataFrame(Cluster3.sum(axis=0)).sort_values(by = 0 , ascending=False)
df4=pd.DataFrame(Cluster4.sum(axis=0)).sort_values(by = 0 , ascending=False)

# Check Cluster 0

In [288]:
df0.head()

,0
Coffee Shop,2.634140
Café,1.780193
Restaurant,1.008395
Italian Restaurant,0.808386
Bar,0.741992


There are many restaurants and bars. It is a night life cluster There are many vanues in general.

# Check Cluster 1

In [290]:
df1.head()

,0
Park,0.50
Playground,0.25
Trail,0.25
Museum,0.00
Mac & Cheese Joint,0.00


There are many outdoor activity, expecially for children. 

# Check Cluster 2

In [291]:
df2.head()

,0
Park,0.647076
Bus Line,0.416667
Lawyer,0.250000
Swim School,0.250000
Sushi Restaurant,0.219298


There are both park and commercial activities and restaurant

# Check Cluster 3

In [292]:
df3.head()

,0
Health & Beauty Service,0.333333
Pool,0.333333
Garden,0.333333
Performing Arts Venue,0.000000
Movie Theater,0.000000


Not many venues. No bar or restaurant 

 # Check Cluster 4

In [293]:
df4.head()

,0
Playground,0.5
Gym,0.5
Museum,0.0
Mac & Cheese Joint,0.0
Market,0.0


Not many vanues. 